In [1]:
import pandas as pd
import dill

from multiprocessing import shared_memory
from fb_shared_memory import FbSharedMemory

In [2]:
# Get the bytes for the dataframe serialized by dill.
shm_b = shared_memory.SharedMemory("dill_shm")
df_bytes = shm_b.buf[:63245615]

# Connect to shared memory in host_nb session.
shm = FbSharedMemory()

In [3]:
%%time
# De-serializing the dataframe with dill from shared memory
df = dill.loads(df_bytes)
df.head()

CPU times: user 87.9 ms, sys: 89.5 ms, total: 177 ms
Wall time: 177 ms


,Unnamed: 0,tweet_id,tweet_created_at,language,user_screen_name,user_created_at,user_id,followers_count,friends_count,time_zone,utc_offset,retweeted_status,retweet_id,retweet_user_screen_name,retweet_user_id
0,0,1068223635024941056,2018-11-29 19:22:15,en,JGuerreroN13,2017-06-07 20:21:48,872549171680694272,245,419,NaN,NaN,1,1.068067e+18,UNFCCC,1.746392e+07
1,1,1068223643765940226,2018-11-29 19:22:17,en,t_rustyboy1720,2017-09-23 03:53:34,911438362271031296,63,232,NaN,NaN,1,1.068211e+18,saskboy,6.634632e+06
2,2,1068223645330432001,2018-11-29 19:22:18,en,MissCJWright,2009-03-15 16:24:32,24544855,1383,4585,NaN,NaN,1,1.068223e+18,EcoSenseNow,1.392956e+09
3,3,1068223649222664194,2018-11-29 19:22:19,en,al_f,2009-07-17 00:05:50,57490401,333,377,NaN,NaN,1,1.068182e+18,ClimateComms,3.626422e+08
4,4,1068223660299677696,2018-11-29 19:22:21,en,Hawknana_,2013-12-21 03:16:04,2255917724,1751,3409,NaN,NaN,1,1.068187e+18,PortlandBEM,1.803082e+07


In [4]:
%%time
# Reading dataframe head from flatbuffer.
shm.dataframe_head("df1", 5)

CPU times: user 15.7 ms, sys: 31.3 ms, total: 47 ms
Wall time: 46.3 ms


,Unnamed: 0,tweet_id,tweet_created_at,language,user_screen_name,user_created_at,user_id,followers_count,friends_count,time_zone,utc_offset,retweeted_status,retweet_id,retweet_user_screen_name,retweet_user_id
0,0,1068223635024941056,2018-11-29 19:22:15,en,JGuerreroN13,2017-06-07 20:21:48,872549171680694272,245,419,NaN,NaN,1,1.068067e+18,UNFCCC,1.746392e+07
1,1,1068223643765940226,2018-11-29 19:22:17,en,t_rustyboy1720,2017-09-23 03:53:34,911438362271031296,63,232,NaN,NaN,1,1.068211e+18,saskboy,6.634632e+06
2,2,1068223645330432001,2018-11-29 19:22:18,en,MissCJWright,2009-03-15 16:24:32,24544855,1383,4585,NaN,NaN,1,1.068223e+18,EcoSenseNow,1.392956e+09
3,3,1068223649222664194,2018-11-29 19:22:19,en,al_f,2009-07-17 00:05:50,57490401,333,377,NaN,NaN,1,1.068182e+18,ClimateComms,3.626422e+08
4,4,1068223660299677696,2018-11-29 19:22:21,en,Hawknana_,2013-12-21 03:16:04,2255917724,1751,3409,NaN,NaN,1,1.068187e+18,PortlandBEM,1.803082e+07


In [5]:
%%time
# De-serializing the dataframe with dill from shared memory
df = dill.loads(df_bytes)
df.groupby("user_id").agg({'followers_count': 'sum'}).head()

CPU times: user 157 ms, sys: 87.2 ms, total: 244 ms
Wall time: 243 ms


,followers_count
user_id,
767,136703
1968,5031
1999,1803
2654,6460
2737,675


In [6]:
%%time
# Performing groupby directly on flatbuffer
shm.dataframe_group_by_sum("df1", "user_id", "followers_count").head()

CPU times: user 4.62 s, sys: 102 ms, total: 4.72 s
Wall time: 4.72 s


,followers_count
user_id,
767,136703
1968,5031
1999,1803
2654,6460
2737,675


In [7]:
%%time
# De-serializing the dataframe, making the change, then re-serializing with dill
df = dill.loads(df_bytes)
df["followers_count"] = df["followers_count"].apply(lambda x: x * 2)
df_bytes = dill.dumps(df)

CPU times: user 4.76 s, sys: 217 ms, total: 4.98 s
Wall time: 4.97 s


In [8]:
%%time
# Performing groupby directly on flatbuffer
shm.dataframe_map_numeric_column("df1", "followers_count", lambda x: x * 2)

CPU times: user 237 ms, sys: 2.56 ms, total: 239 ms
Wall time: 238 ms
